In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.font_manager as fm
import koreanize_matplotlib

# 폰트 경로 확인 및 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  # 설치된 폰트 경로
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

# 한글 깨짐 방지 - 마이너스 기호 처리
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# 서비스 계정 키 경로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/home/eunyu/sprintda05-eunyu-a9946f7dea9c.json"

# GCS의 Parquet 파일 경로
gcs_path = 'gs://final_project_enuyu/data/final_project/votes/accounts_school.parquet'

# 데이터 불러오기
school_df = pd.read_parquet(gcs_path, engine='pyarrow')
school_df

,id,address,student_count,school_type
0,4,충청북도 충주시,239,H
1,5,충청북도 충주시,160,M
2,6,충청북도 충주시,200,H
3,7,충청북도 충주시,114,H
4,8,충청북도 충주시,139,M
...,...,...,...,...
5946,5960,경상남도 양산시,166,H
5947,5961,인천광역시 서구,77,M
5948,5963,경북 김천시,17,H
5949,5964,-,1,H


In [3]:
school_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5951 entries, 0 to 5950
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             5951 non-null   int64 
 1   address        5951 non-null   object
 2   student_count  5951 non-null   int64 
 3   school_type    5951 non-null   object
dtypes: int64(2), object(2)
memory usage: 186.1+ KB


In [4]:
school_df.isnull().sum()

id               0
address          0
student_count    0
school_type      0
dtype: int64

In [ ]:
school_df.describe(include='all')

,id,address,student_count,school_type
count,5951.000000,5951,5951.000000,5951
unique,NaN,278,NaN,2
top,NaN,경기도 화성시,NaN,M
freq,NaN,78,NaN,3290
mean,2981.026046,NaN,113.772979,NaN
std,1719.084020,NaN,102.863428,NaN
min,4.000000,NaN,0.000000,NaN
25%,1493.500000,NaN,16.000000,NaN
50%,2981.000000,NaN,97.000000,NaN
75%,4469.500000,NaN,183.000000,NaN


#### 학교id 고유 5951

In [7]:
# qusetion_record에서 참여한 유저 수
school_df['id'].nunique()

5951

#### student_count = 0인 학교 확인

In [6]:
# student_count가 0인 학교 필터링
zero_students = school_df[school_df['student_count'] == 0]
print(f"학생 수가 0명인 학교 수: {len(zero_students)}")
zero_students['school_type'].value_counts()
zero_students['address'].value_counts().head(10)

학생 수가 0명인 학교 수: 387


address
전라남도 신안군    8
경상북도 청송군    7
강원도 영월군     7
경상북도 영천시    6
강원도 삼척시     6
충청북도 제천시    6
경상북도 상주시    5
전라북도 익산시    5
경상북도 의성군    5
경상남도 진주시    5
Name: count, dtype: int64

#### school_type별 평균 학생 수

In [7]:
# school_type별 student_count 평균
school_type_avg = school_df.groupby('school_type')['student_count'].mean()
print("school_type별 평균 학생 수:")
print(school_type_avg)

school_type별 평균 학생 수:
school_type
H    141.349117
M     91.468997
Name: student_count, dtype: float64


#### 지역별 학교 수 분포 (address)

In [8]:
# address별 학교 수 카운트
address_count = school_df['address'].value_counts().head(10)
print("학교 수가 많은 상위 10개 지역:")
print(address_count)

학교 수가 많은 상위 10개 지역:
address
경기도 화성시        78
경기도 부천시        60
경상남도 김해시       58
경기도 남양주시       58
대구광역시 달서구      56
서울특별시 노원구      54
경기도 성남시 분당구    53
서울특별시 송파구      52
제주특별자치도 제주시    52
광주광역시 북구       52
Name: count, dtype: int64


#### 이상치 탐지 (학생 수가 500명 이상인 학교)

In [11]:
# student_count가 500명 이상인 경우 필터링
outliers = school_df[school_df['student_count'] >= 500]
print(f"학생 수가 500명 이상인 학교 수: {len(outliers)}")
outliers[['id', 'address', 'student_count', 'school_type']].sort_values(by='student_count', ascending=False).head()

학생 수가 500명 이상인 학교 수: 5


,id,address,student_count,school_type
365,369,충청남도 아산시,578,H
1713,1719,울산광역시 울주군,550,H
4509,4516,경기도 화성시,507,H
5365,5372,경기도 남양주시,505,H
5513,5520,경기도 광명시,500,M
